In [1]:
import pandas as pd
import numpy as np

import keras as keras
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Conv1D, Flatten, Dropout, GRU, Input, SimpleRNN
from tensorflow.keras.layers import AveragePooling1D, MaxPool1D, GlobalMaxPool1D, AveragePooling1D, GlobalAveragePooling1D

import pickle
from keras_pickle_wrapper import KerasPickleWrapper

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
STOPWORDS = set(stopwords.words('english'))


Using TensorFlow backend.


In [2]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16675666159517920964
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5077532672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11129327609517771562
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
news1 = pd.read_csv('./data/djia_news copy.csv')
news2 = pd.read_csv('./data/nasdaq.csv')
combined_news = news1.append(news2)
combined_news = combined_news[['Label', 'Headline']]

print(combined_news.head())

   Label                                           Headline
0      0  Employer who stole nearly $3M in wages from 15...
1      1  Huge new Facebook data leak exposed intimate d...
2      0  A campaign has accelerated to turn a disused r...
3      1  Google launches global human trafficking helpl...
4      1  Over 3m Saudi Women Don’t Have ID Cards; Saudi...


In [4]:
# ps = PorterStemmer()
# lamatizer = WordNetLemmatizer()
# combined_news['Headline'] = combined_news['Headline'].replace('[^\s\w]', '')
# combined_news['Headline_token'] = combined_news['Headline'].apply(lambda x: word_tokenize(x))
# combined_news['Headline_recon'] = combined_news['Headline_token'].apply(lambda x: list(ps.stem(i) for i in x))
# combined_news['Headline_recon'] = combined_news['Headline_recon'].apply(lambda x: ' '.join(list(i for i in x if i not in STOPWORDS)))
# print(combined_news['Headline_recon'])

In [5]:
combined_news['Label'].value_counts()

0    9995
1    5236
2     331
Name: Label, dtype: int64

In [6]:
#Resampling due to imbalanced dataset
class_count_0, class_count_1, class_count_2 = combined_news['Label'].value_counts()

#Seperating the classes
class_0 = combined_news[combined_news['Label'] == 0]
class_1 = combined_news[combined_news['Label'] == 1]
class_2 = combined_news[combined_news['Label'] == 2]
print(class_0.shape, class_1.shape, class_2.shape)

(9995, 2) (5236, 2) (331, 2)


In [7]:
# #Random Under Sampling
# class_0_under = class_0.sample(class_count_2)
# class_1_under = class_1.sample(class_count_2)

# train_under = pd.concat([class_0_under, class_1_under, class_2], axis = 0)

# train_under['Label'].value_counts().plot(kind = 'bar', title = 'count (target)')

In [8]:
# #Random Over Sampling
# class_2_over = class_2.sample(class_count_0, replace = True)

# train_over = pd.concat([class_0, class_1, class_2_over], axis = 0)
# train_over['Label'].value_counts().plot(kind = 'bar', title = 'count (target)')

In [9]:
#Getting neutral data from another dataset
news3 = pd.read_csv('./data/train.csv')
news3 = news3[news3['Label'] == 'neutral']
news3 = news3.sample(class_count_0)
news3['Label'] = news3['Label'].replace('neutral', 2)
combined_news = combined_news.append(news3)

In [10]:
combined_news['Headline'] = combined_news['Headline'].astype(str)
combined_news['Label'] = combined_news['Label'].astype(int)

In [11]:
combined_news['Label'].value_counts()

2    10326
0     9995
1     5236
Name: Label, dtype: int64

In [12]:
#This is important for splitting a one column classification to an n-column classification
y = pd.get_dummies(combined_news['Label'])
# y.shape

In [13]:
train_text, test_text, train_labels, test_labels = train_test_split(combined_news['Headline'].to_numpy(), y.to_numpy(), test_size = 0.2, random_state = 42)

In [14]:
print(len(test_text),len(train_text))

5112 20445


In [15]:
type(train_text)

numpy.ndarray

In [16]:
vocab_size = 10000
emb_size = 300
max_length = 20000
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<oov>'

In [17]:
import re
import string

In [18]:
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for i in range(0, len(train_text)):
    temp = train_text[i]
    temp = re.sub('@\S+', '', temp)
    temp = re.sub('https*\S+', '', temp)
    temp = re.sub('#\S+', '', temp)
    temp = re.sub("[^\s\w]", '', temp)
    temp = re.sub("\'\w+", '', temp)
    temp = re.sub(r'\w*\d+\w*', '', temp)
    temp = re.sub('\s{2,}', '', temp)
    
    temp = temp.split()
    
    temp = [stemmer.lemmatize(word) for word in temp]
    temp = ' '.join(temp)
    train_text[i] = temp

for i in range(0, len(test_text)):
    temp = train_text[i]
    temp = re.sub('@\S+:', '', temp)
    temp = re.sub('https*\S+', '', temp)
    temp = re.sub("[^\s\w]", '', temp)
    temp = re.sub('#\S+', '', temp)
    temp = re.sub("\'\w+", '', temp)
    temp = re.sub(r'\w*\d+\w*', '', temp)
    temp = re.sub('\s{2,}', '', temp)
    temp = temp.split()
    
    temp = [stemmer.lemmatize(word) for word in temp]
    temp = ' '.join(temp)
    test_text[i] = temp

In [19]:
# filtered_train_text = []
# for w in range(0, len(train_text)):
#     if train_text[w] not in STOPWORDS:
#         filtered_train_text.append(train_text[w])

# filtered_test_text = []
# for w in range(0, len(test_text)):
#     if test_text[w] not in STOPWORDS:
#         filtered_test_text.append(test_text[w])

# len(filtered_train_text), len(filtered_test_text)

(20439, 5111)

In [20]:
tokenizer = Tokenizer(num_words = vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(train_text)

word_index = tokenizer.word_index
print(len(word_index))

train_sequences =  tokenizer.texts_to_sequences(train_text)
train_padded = pad_sequences(train_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(test_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

33775


In [21]:
# import tensorflow_hub as hub

In [22]:
# embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [23]:
# train_embed = embed(train_text)
# test_embed = embed(test_text)
# len(train_embed[0])

In [24]:
# #Another embedding method
# embedding_index = {}
# glove_file = open('./glove.6B.300d.txt', encoding="utf8")
# for line in glove_file:
#     data = line.split()
#     word = data[0]
#     vector = np.asarray(data[1:], dtype = 'float32')
#     embedding_index[word] = vector
# glove_file.close()

# len(embedding_index)

In [25]:
# embedding_matrix = np.zeros((len(word_index) + 1, emb_size))
# for word in word_index.items():
#     embedding_vector = embedding_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

# embedding_matrix.shape
# embedding_matrix = embedding_matrix[:vocab_size][:]

In [29]:
model = Sequential()
model.add(Embedding(vocab_size, emb_size,input_length = max_length))
model.add(LSTM(16,return_sequences= True))
model.add(Bidirectional(LSTM(16,return_sequences= True)))
model.add(GlobalMaxPool1D())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(3,activation = 'softmax'))
          
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20000, 300)        3000000   
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               154112    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_11 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_12 (Dense)             (None, 64)               

In [30]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [31]:
with tf.device('/device:GPU:0'):
    history = model.fit(train_padded,
                        train_labels,
                        shuffle = True,
                        epochs = 4,
                        validation_data = (test_padded, test_labels))

Train on 20445 samples, validate on 5112 samples
Epoch 1/4
20445/20445 [==============================] - 219s 11ms/sample - loss: 0.6017 - accuracy: 0.7153 - val_loss: 3.1708 - val_accuracy: 0.3995
Epoch 2/4
20445/20445 [==============================] - 219s 11ms/sample - loss: 0.4356 - accuracy: 0.7807 - val_loss: 4.3779 - val_accuracy: 0.3599
Epoch 3/4
20445/20445 [==============================] - 218s 11ms/sample - loss: 0.3375 - accuracy: 0.8532 - val_loss: 4.7966 - val_accuracy: 0.3550
Epoch 4/4
 1120/20445 [>.............................] - ETA: 3:05 - loss: 0.2012 - accuracy: 0.9338

KeyboardInterrupt: 

In [ ]:
def plot_metrics(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_' + metric])
    plt.show()
plot_metrics(history, 'accuracy')
plot_metrics(history, 'loss')

In [ ]:
import pickle
from keras_pickle_wrapper import KerasPickleWrapper

pickled_wrapped = KerasPickleWrapper(model)
filename = './model_weight.pkl'
pickle.dump(pickled_wrapped, open(filename, 'wb'))